In [53]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier as KNN

class Solution():    
    #structure of the solution 
    def __init__(self):
        self.num_features = None
        self.num_agents = None
        self.max_iter = None
        self.obj_function = None
        self.execution_time = None
        self.convergence_curve = {}
        self.best_agent = None
        self.best_fitness = None
        self.best_accuracy = None
        self.final_population = None
        self.final_fitness = None
        self.final_accuracy = None


class Data():
    # structure of the training data
    def __init__(self):
        self.train_X = None
        self.train_Y = None
        self.val_X = None
        self.val_Y = None



def initialize(num_agents, num_features):
    # define min and max number of features
    min_features = int(0.3 * num_features)
    max_features = int(0.6 * num_features)

    # initialize the agents with zeros
    agents = np.zeros((num_agents, num_features))

    # select random features for each agent
    for agent_no in range(num_agents):

        # find random indices
        cur_count = np.random.randint(min_features, max_features)
        temp_vec = np.random.rand(1, num_features)
        temp_idx = np.argsort(temp_vec)[0][0:cur_count]

        # select the features with the ranom indices
        agents[agent_no][temp_idx] = 1   

    return agents



def sort_agents(agents, obj, data):
    # sort the agents according to fitness
    train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        reduced_agents= sorted_agents[:len(sorted_agents)//2]
        reduced_fitness = sorted_fitness[:len(sorted_fitness)//2]
    return reduced_agents, reduced_fitness



def display(agents, fitness, agent_name='Agent'):
    # display the population
    print('\nNumber of agents: {}'.format(agents.shape[0]))
    print('\n------------- Best Agent ---------------')
    print('Fitness: {}'.format(fitness[0]))
    print('Number of Features: {}'.format(int(np.sum(agents[0]))))
    print('----------------------------------------\n')

    for id, agent in enumerate(agents):
        print('{} {} - Fitness: {}, Number of Features: {}'.format(agent_name, id+1, fitness[id], int(np.sum(agent))))

    print('================================================================================\n')


#score=fitness_score_values(data,target)
#print(score)
def compute_accuracy(agent, train_X, test_X, train_Y, test_Y): 
    # compute classification accuracy of the given agents
    fitness=0
    number=0
    for i in range(len(agent)):
        if agent[i]==1:
            number=number+1
            fitness=fitness+score[i]
    return fitness/number        

def DP(chromosomes,num_agents, num_features):
    opposite_chromosomes=[]
    #print(chromosomes)
    for i in range(len(chromosomes)):
        opp=[0 for i in range(num_features)]
        #print(i)
        for j in range(len(chromosomes[i])):
            #opp.append(int(chromosomes[i][j])^1)
            if chromosomes[i][j]==1:
                opp[independent_features[j]]=1
            else:
                opp[j]=1
                
        opposite_chromosomes.append(opp)
    return opposite_chromosomes    
            
def merge(chromosomes,opposition_chromosomes,num_agents, num_features):
    merge_chromosomes=[]
    for i in range(len(chromosomes)):
        merge_chromosomes.append(chromosomes[i])
    for i in range(len(opposition_chromosomes)):
        merge_chromosomes.append(opposition_chromosomes[i])
    return np.asarray(merge_chromosomes)    
        
        
    

def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.9):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.9

    weight_feat = 1 - weight_acc
    num_features = agent.shape[0]
    
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    
    return fitness

In [54]:
import numpy as np
import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import datasets


def GA(num_agents, max_iter, train_data, train_label, obj_function=compute_fitness, prob_cross=0.4, prob_mut=0.3, save_conv_graph=False):

    # Genetic Algorithm
    ############################### Parameters ####################################
    #                                                                             #
    #   num_agents: number of chromosomes                                         #
    #   max_iter: maximum number of generations                                   #
    #   train_data: training samples of data                                      #
    #   train_label: class labels for the training samples                        #                
    #   obj_function: the function to maximize while doing feature selection      #
    #   prob_cross: probability of crossover                                      #
    #   prob_mut: probability of mutation                                         #
    #   save_conv_graph: boolean value for saving convergence graph               #
    #                                                                             #
    ###############################################################################

    short_name = 'GA'
    agent_name = 'Chromosome'
    train_data, train_label = np.array(train_data), np.array(train_label)
    num_features = train_data.shape[1]
    cross_limit = 5

    # setting up the objectives
    weight_acc = None
    if(obj_function==compute_fitness):
        weight_acc = float(input('Weight for the classification accuracy [0-1]: '))
    obj = (obj_function, weight_acc)
    compute_accuracy = (compute_fitness, 1) # compute_accuracy is just compute_fitness with accuracy weight as 1

    # initialize chromosomes and Leader (the agent with the max fitness)
    chromosomes = initialize(num_agents, num_features)
    fitness = np.zeros(num_agents)
    accuracy = np.zeros(num_agents)
    Leader_agent = np.zeros((1, num_features))
    Leader_fitness = float("-inf")
    Leader_accuracy = float("-inf")

    # initialize convergence curves
    convergence_curve = {}
    convergence_curve['fitness'] = np.zeros(max_iter)
    convergence_curve['feature_count'] = np.zeros(max_iter)

    # initialize data class
    data = Data()
    val_size = float(input('Enter the percentage of data wanted for valdiation [0, 100]: '))/100
    data.train_X, data.val_X, data.train_Y, data.val_Y = train_test_split(train_data, train_label, stratify=train_label, test_size=val_size)

    # create a solution object
    solution = Solution()
    solution.num_agents = num_agents
    solution.max_iter = max_iter
    solution.num_features = num_features
    solution.obj_function = obj_function

    # rank initial population
    chromosomes, fitness = sort_agents(chromosomes, obj, data)

    # start timer
    start_time = time.time()
    
    # main loop
    for iter_no in range(max_iter):
        print('\n================================================================================')
        print('                          Iteration - {}'.format(iter_no+1))
        print('================================================================================\n')
        
        #opposition based population of chromosomes
        opposition_chromosomes=DP(chromosomes,num_agents, num_features)
        
        # perform crossover, mutation and replacement
        cross_mut(chromosomes, fitness, obj_function, data, prob_cross, cross_limit, prob_mut)

        #merge both the set of chromosomes
        chromosomes= merge(chromosomes,opposition_chromosomes,num_agents, num_features)
        
        # update final information
        chromosomes, fitness = sort_agents(chromosomes, obj, data)
        display(chromosomes, fitness, agent_name)
        if fitness[0]>Leader_fitness:
            Leader_agent = chromosomes[0].copy()
            Leader_fitness = fitness[0].copy()
        convergence_curve['fitness'][iter_no] = Leader_fitness
        convergence_curve['feature_count'][iter_no] = int(np.sum(Leader_agent))

    # compute final accuracy
    Leader_agent, Leader_accuracy = sort_agents(Leader_agent, compute_accuracy, data)
    chromosomes, accuracy = sort_agents(chromosomes, compute_accuracy, data)

    print('\n================================================================================')
    print('                                    Final Result                                  ')
    print('================================================================================\n')
    print('Leader ' + agent_name + ' Dimension : {}'.format(int(np.sum(Leader_agent))))
    print('Leader ' + agent_name + ' Fitness : {}'.format(Leader_fitness))
    
    print('\n================================================================================\n')

    # stop timer
    end_time = time.time()
    exec_time = end_time - start_time
    
    

    # update attributes of solution
    solution.best_agent = Leader_agent
    solution.best_fitness = Leader_fitness
    solution.best_accuracy = Leader_accuracy
    solution.convergence_curve = convergence_curve
    solution.final_population = chromosomes
    solution.final_fitness = fitness
    solution.final_accuracy = accuracy
    solution.execution_time = exec_time

    return solution


def crossover(parent_1, parent_2, prob_cross):
    # perform crossover with crossover probability prob_cross
    num_features = parent_1.shape[0]
    child_1 = parent_1.copy()
    child_2 = parent_2.copy()

    for i in range(num_features):
        if(np.random.rand()<prob_cross):
            child_1[i] = parent_2[i]
            child_2[i] = parent_1[i]

    return child_1, child_2


def mutation(chromosome, prob_mut):
    # perform mutation with mutation probability prob_mut
    num_features = chromosome.shape[0]
    mut_chromosome = chromosome.copy()

    for i in range(num_features):
        if(np.random.rand()<prob_mut):
            mut_chromosome[i] = 1-mut_chromosome[i]
    
    return mut_chromosome


def roulette_wheel(fitness):
    # Perform roulette wheel selection
    maximum = sum([f for f in fitness])
    selection_probs = [f/maximum for f in fitness]
    return np.random.choice(len(fitness), p=selection_probs)


def cross_mut(chromosomes, fitness, obj_function, data, prob_cross, cross_limit, prob_mut):
    # perform crossover, mutation and replacement
    count = 0
    num_agents = chromosomes.shape[0]
    train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    print('Crossover-Mutation phase starting....')

    while(count<cross_limit):
        print('\nCrossover no. {}'.format(count+1))
        id_1 = roulette_wheel(fitness)
        id_2 = roulette_wheel(fitness)

        if(id_1 != id_2):
            child_1, child_2 = crossover(chromosomes[id_1], chromosomes[id_2], prob_cross)
            child_1 = mutation(child_1, prob_mut)
            child_2 = mutation(child_2, prob_mut)
            fitness_1 = obj_function(child_1, train_X, val_X, train_Y, val_Y)
            fitness_2 = obj_function(child_2, train_X, val_X, train_Y, val_Y)

            if(fitness_1 < fitness_2):
                temp = child_1, fitness_1
                child_1, fitness_1 = child_2, fitness_2
                child_2, fitness_2 = temp

            for i in range(num_agents):
                if(fitness_1 > fitness[i]):
                    print('1st child replaced with chromosome having id {}'.format(i+1))
                    chromosomes[i] = child_1
                    fitness[i] = fitness_1
                    break

            for i in range(num_agents):
                if(fitness_2 > fitness[i]):
                    print('2nd child replaced with chromosome having id {}'.format(i+1))
                    chromosomes[i] = child_2
                    fitness[i] = fitness_2
                    break

            count = count+1

        else:
            print('Crossover failed....')
            print('Restarting crossover....\n')


In [20]:
import pandas as pd
df=pd.read_csv('LtrP_U_Smote.csv')
(a,b)=df.shape
print(a,b)
target=df['attr_768']
data=df[df.columns.drop('attr_768')]
for i in range(len(target)):
    target[i]=target[i][1]
for i in range(len(target)):
    target[i]=int(target[i])
import numpy as np
target=np.array(target)
#print(target)
data=np.array(data)
print(data)

target=target.astype('int')

455 768


c:\users\user1\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\user1\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


[[0.376976   0.119887   0.030786   ... 0.012032   0.014866   0.122389  ]
 [0.332956   0.14643    0.029162   ... 0.012266   0.016299   0.132057  ]
 [0.223374   0.277216   0.024647   ... 0.010346   0.012712   0.190544  ]
 ...
 [0.24057218 0.34249409 0.02036273 ... 0.01048694 0.01408021 0.26175596]
 [0.20152361 0.3360178  0.02105929 ... 0.0118019  0.01597291 0.14756469]
 [0.21470849 0.31991871 0.02259656 ... 0.01181161 0.01646684 0.13668292]]


In [32]:
def fitness_score_values(data,target):
    from Py_FS.filter import PCC as PCC
    from Py_FS.filter import MI as MI
    from sklearn.feature_selection import mutual_info_classif
    from Py_FS.filter import Relief as Relief
    from sklearn.feature_selection import chi2
    
    #print('Pcc scores')
    #PCC_solution = PCC(data, target)
    #print(PCC_solution.scores)
    print('MI scores')
    mi_scores=mutual_info_classif(data, target, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
    print((mi_scores))
    print('Relief scores')
    Relief_solution = Relief(data, target)
    print((Relief_solution.scores))
    print('chi sqaure scores')
    chi_scores = chi2(data,target)
    max_score=max(chi_scores[0])
    for i in range(len(chi_scores[0])):
        chi_scores[0][i]=chi_scores[0][i]/max_score
    print((chi_scores[0]))
    shapley=[]
    for i in range(b-1):
        shapley.append((mi_scores[i]+chi_scores[0][i]+Relief_solution.scores[i])/3)
    return shapley

In [45]:
def PCC_features():
    from scipy import stats
    independent_features=[]
    for i in range(b-1):
        x=np.asarray(df['attr_'+str(i+1)])
        mmin=10000
        position=-1
        for j in range(b-1):
            y=np.asarray(df['attr_'+str(j+1)])
            corr,_=stats.pearsonr(x,y)
            if corr<mmin:
                mmin=corr
                position=j
        independent_features.append(position)
    return independent_features    

In [33]:
#fitness_score_values
score=fitness_score_values(data,target)
print(score)

MI scores
[0.62086555 0.72161957 0.65410147 0.73758717 0.65768311 0.53186139
 0.80461648 0.74789711 0.58117686 0.75653618 0.42555412 0.31521261
 0.61737465 0.75221777 0.4834012  0.5553291  0.44086578 0.62098974
 0.77125633 0.56395013 0.42427566 0.49637547 0.61330961 0.42424438
 0.78881868 0.62756464 0.49877316 0.6260716  0.56238905 0.61449506
 0.4816571  0.70430688 0.59682382 0.56569457 0.53645731 0.44986761
 0.59447575 0.67305856 0.45528054 0.43327362 0.61014211 0.61165498
 0.65518333 0.46926065 0.39977497 0.48278492 0.60375397 0.56796554
 0.33082786 0.5863025  0.52119989 0.76321158 0.49269705 0.72213704
 0.75412493 0.62428509 0.90899512 0.60632407 0.70948238 0.73994707
 0.71953895 0.49395448 0.5606041  0.72207867 0.53610989 0.81564825
 0.58469591 0.56233802 0.66969086 0.49911259 0.40809983 0.56781826
 0.56383339 0.40319543 0.39375474 0.56667155 0.61104325 0.64481512
 0.60076098 0.50042573 0.44354223 0.3984889  0.61977387 0.76190426
 0.66728707 0.58655563 0.56495608 0.61007452 0.59204

[0.03921569 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.05882353 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.03921569 0.15686275 0.03921569
 0.03921569 0.03921569 0.03921569 0.03921569 0.05882353 0.07843137
 0.15686275 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569
 0.09803922 0.03921569 0.03921569 0.05882353 0.11764706 0.07843137
 0.03921569 0.03921569 0.07843137 0.07843137 0.03921569 0.11764706
 0.03921569 0.15686275 0.07843137 0.03921569 0.07843137 0.03921569
 0.03921569 0.15686275 0.09803922 0.03921569 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569 0.03921569
 0.03921569 0.07843137 0.03921569 0.11764706 0.03921569 0.03921569
 0.03921569 0.03921569 0.03921569 0.23529412 0.03921569 0.03921569
 0.03921569 0.01960784 0.03921569 0.11764706 0.05882353 0.0392

In [46]:

independent_features=PCC_features()
print(independent_features)

[211, 241, 591, 766, 219, 591, 383, 383, 57, 573, 59, 0, 591, 560, 573, 236, 725, 206, 219, 2, 177, 59, 725, 748, 573, 206, 0, 59, 708, 299, 573, 442, 366, 0, 59, 59, 725, 560, 51, 354, 590, 709, 299, 716, 182, 177, 709, 725, 536, 59, 120, 299, 59, 12, 357, 5, 357, 357, 312, 576, 356, 270, 235, 235, 729, 235, 206, 119, 29, 766, 766, 550, 51, 29, 64, 766, 235, 383, 383, 0, 0, 766, 1, 219, 383, 383, 405, 766, 509, 119, 58, 383, 383, 61, 474, 124, 589, 42, 130, 766, 82, 481, 766, 130, 130, 82, 82, 766, 766, 474, 481, 766, 766, 155, 766, 481, 418, 82, 414, 356, 1, 296, 206, 1, 442, 206, 58, 115, 725, 0, 511, 51, 560, 177, 474, 296, 219, 0, 130, 0, 474, 178, 265, 383, 474, 130, 64, 124, 119, 560, 474, 405, 474, 474, 474, 115, 29, 130, 766, 418, 481, 725, 141, 0, 418, 148, 0, 766, 766, 481, 0, 766, 474, 436, 481, 241, 82, 20, 241, 414, 766, 219, 748, 206, 206, 573, 560, 23, 573, 591, 219, 51, 716, 59, 234, 219, 543, 354, 716, 533, 296, 206, 206, 0, 236, 194, 357, 587, 29, 543, 0, 0, 120, 357

In [56]:

solution = GA(num_agents=30, max_iter=30, train_data=data,  prob_cross=0.6, prob_mut=0.4,train_label=target,save_conv_graph=True)

Weight for the classification accuracy [0-1]: 0.9
Enter the percentage of data wanted for valdiation [0, 100]: 10

                          Iteration - 1

Crossover-Mutation phase starting....

Crossover no. 1
1st child replaced with chromosome having id 11
2nd child replaced with chromosome having id 13

Crossover no. 2
1st child replaced with chromosome having id 12

Crossover no. 3
1st child replaced with chromosome having id 10

Crossover no. 4
1st child replaced with chromosome having id 12

Crossover no. 5
1st child replaced with chromosome having id 13

Number of agents: 15

------------- Best Agent ---------------
Fitness: 0.27087404620637645
Number of Features: 244
----------------------------------------

Chromosome 1 - Fitness: 0.27087404620637645, Number of Features: 244
Chromosome 2 - Fitness: 0.26720054418859585, Number of Features: 230
Chromosome 3 - Fitness: 0.26584101224796153, Number of Features: 239
Chromosome 4 - Fitness: 0.2644635654170702, Number of Features: 271


Number of agents: 15

------------- Best Agent ---------------
Fitness: 0.27087404620637645
Number of Features: 244
----------------------------------------

Chromosome 1 - Fitness: 0.27087404620637645, Number of Features: 244
Chromosome 2 - Fitness: 0.26720054418859585, Number of Features: 230
Chromosome 3 - Fitness: 0.26584101224796153, Number of Features: 239
Chromosome 4 - Fitness: 0.2644635654170702, Number of Features: 271
Chromosome 5 - Fitness: 0.26415287620145167, Number of Features: 267
Chromosome 6 - Fitness: 0.263031190631334, Number of Features: 294
Chromosome 7 - Fitness: 0.2613817270371704, Number of Features: 338
Chromosome 8 - Fitness: 0.25866461742421076, Number of Features: 350
Chromosome 9 - Fitness: 0.2585906816165079, Number of Features: 334
Chromosome 10 - Fitness: 0.25627273303527215, Number of Features: 344
Chromosome 11 - Fitness: 0.25590927734155144, Number of Features: 349
Chromosome 12 - Fitness: 0.2552559718283308, Number of Features: 349
Chromosome 13 - 

Crossover-Mutation phase starting....

Crossover no. 1
1st child replaced with chromosome having id 11

Crossover no. 2
1st child replaced with chromosome having id 12

Crossover no. 3

Crossover no. 4

Crossover no. 5

Number of agents: 15

------------- Best Agent ---------------
Fitness: 0.27087404620637645
Number of Features: 244
----------------------------------------

Chromosome 1 - Fitness: 0.27087404620637645, Number of Features: 244
Chromosome 2 - Fitness: 0.26720054418859585, Number of Features: 230
Chromosome 3 - Fitness: 0.26584101224796153, Number of Features: 239
Chromosome 4 - Fitness: 0.2644635654170702, Number of Features: 271
Chromosome 5 - Fitness: 0.26415287620145167, Number of Features: 267
Chromosome 6 - Fitness: 0.263031190631334, Number of Features: 294
Chromosome 7 - Fitness: 0.2613817270371704, Number of Features: 338
Chromosome 8 - Fitness: 0.25866461742421076, Number of Features: 350
Chromosome 9 - Fitness: 0.2585906816165079, Number of Features: 334
Chromo

Crossover-Mutation phase starting....

Crossover no. 1

Crossover no. 2
1st child replaced with chromosome having id 10

Crossover no. 3

Crossover no. 4

Crossover no. 5

Number of agents: 15

------------- Best Agent ---------------
Fitness: 0.27087404620637645
Number of Features: 244
----------------------------------------

Chromosome 1 - Fitness: 0.27087404620637645, Number of Features: 244
Chromosome 2 - Fitness: 0.26720054418859585, Number of Features: 230
Chromosome 3 - Fitness: 0.26584101224796153, Number of Features: 239
Chromosome 4 - Fitness: 0.2644635654170702, Number of Features: 271
Chromosome 5 - Fitness: 0.26415287620145167, Number of Features: 267
Chromosome 6 - Fitness: 0.263031190631334, Number of Features: 294
Chromosome 7 - Fitness: 0.2613817270371704, Number of Features: 338
Chromosome 8 - Fitness: 0.25866461742421076, Number of Features: 350
Chromosome 9 - Fitness: 0.2585906816165079, Number of Features: 334
Chromosome 10 - Fitness: 0.2578459840170821, Number of


Crossover no. 4

Crossover no. 5

Number of agents: 15

------------- Best Agent ---------------
Fitness: 0.27087404620637645
Number of Features: 244
----------------------------------------

Chromosome 1 - Fitness: 0.27087404620637645, Number of Features: 244
Chromosome 2 - Fitness: 0.26720054418859585, Number of Features: 230
Chromosome 3 - Fitness: 0.26584101224796153, Number of Features: 239
Chromosome 4 - Fitness: 0.2644635654170702, Number of Features: 271
Chromosome 5 - Fitness: 0.26415287620145167, Number of Features: 267
Chromosome 6 - Fitness: 0.263031190631334, Number of Features: 294
Chromosome 7 - Fitness: 0.2613817270371704, Number of Features: 338
Chromosome 8 - Fitness: 0.25866461742421076, Number of Features: 350
Chromosome 9 - Fitness: 0.2585906816165079, Number of Features: 334
Chromosome 10 - Fitness: 0.25845676367868003, Number of Features: 334
Chromosome 11 - Fitness: 0.25772926580899463, Number of Features: 359
Chromosome 12 - Fitness: 0.25602239129745574, Numbe

In [43]:

x=np.asarray(df['attr_1'])
y=np.asarray(df['attr_2'])
stats.pearsonr(x,y)


(-0.7108670769369573, 3.0409744937529234e-71)

[211,
 241,
 591,
 766,
 219,
 591,
 383,
 383,
 57,
 573,
 59,
 0,
 591,
 560,
 573,
 236,
 725,
 206,
 219,
 2,
 177,
 59,
 725,
 748,
 573,
 206,
 0,
 59,
 708,
 299,
 573,
 442,
 366,
 0,
 59,
 59,
 725,
 560,
 51,
 354,
 590,
 709,
 299,
 716,
 182,
 177,
 709,
 725,
 536,
 59,
 120,
 299,
 59,
 12,
 357,
 5,
 357,
 357,
 312,
 576,
 356,
 270,
 235,
 235,
 729,
 235,
 206,
 119,
 29,
 766,
 766,
 550,
 51,
 29,
 64,
 766,
 235,
 383,
 383,
 0,
 0,
 766,
 1,
 219,
 383,
 383,
 405,
 766,
 509,
 119,
 58,
 383,
 383,
 61,
 474,
 124,
 589,
 42,
 130,
 766,
 82,
 481,
 766,
 130,
 130,
 82,
 82,
 766,
 766,
 474,
 481,
 766,
 766,
 155,
 766,
 481,
 418,
 82,
 414,
 356,
 1,
 296,
 206,
 1,
 442,
 206,
 58,
 115,
 725,
 0,
 511,
 51,
 560,
 177,
 474,
 296,
 219,
 0,
 130,
 0,
 474,
 178,
 265,
 383,
 474,
 130,
 64,
 124,
 119,
 560,
 474,
 405,
 474,
 474,
 474,
 115,
 29,
 130,
 766,
 418,
 481,
 725,
 141,
 0,
 418,
 148,
 0,
 766,
 766,
 481,
 0,
 766,
 474,
 436,
 481,
 241,
 